In [1]:
import os
import re
import json
import zipfile
import sqlite3
from collections import Counter

In [2]:
from tqdm import tqdm

In [3]:
def file_lines():
    with zipfile.ZipFile('dgk_shooter_min.conv.zip', 'r') as archive:
        with archive.open('dgk_shooter_min.conv', 'r') as fp:
            b = fp.read()
    content = b.decode('utf8', 'ignore')
    lines = []
    for line in tqdm(content.split('\n')):
        try:
            line = line.replace('\n', '').strip()
            if line.startswith('E'):
                lines.append('')
            elif line.startswith('M '):
                chars = line[2:].split('/')
                while len(chars) and chars[len(chars) - 1] == '.':
                    chars.pop()
                if chars:
                    sentence = ''.join(chars)
                    sentence = re.sub('\s+', '，', sentence)
                    lines.append(sentence)
        except:
            print(line)
            return lines
            lines.append('')
    return lines

In [4]:
lines = file_lines()

100%|██████████| 4268085/4268085 [00:11<00:00, 385895.08it/s]


In [5]:
print(len(lines))

4268080


In [6]:
lines[:10]

['',
 '畹华吾侄',
 '你接到这封信的时候',
 '不知道大伯还在不在人世了',
 '',
 '咱们梅家从你爷爷起',
 '就一直小心翼翼地唱戏',
 '侍奉宫廷侍奉百姓',
 '从来不曾遭此大祸',
 '太后的万寿节谁敢不穿红']

In [7]:
db = 'conversation.db'
if os.path.exists(db):
    os.remove(db)
conn = sqlite3.connect(db)
cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS conversation
    (ask text, answer text)
    """)
conn.commit()

In [8]:
def contain_chinese(s):
    if re.findall('[\u4e00-\u9fa5]+', s):
        return True
    return False

In [9]:
def combine(a, b):
    if len(a) > 0 and len(b) > 0:
        chinese_punctuation = '。？！，、；：“”‘’（）—'
        english_punctuation = '.,;:!\'"-[](){}…<>/'
        if a[-1:] not in chinese_punctuation and a[-1:] not in english_punctuation:
            return a + '，' + b
        else:
            return a + b
    if len(a) > 0 and len(b) <= 0:
        return a
    if len(a) <= 0 and len(b) > 0:
        return b
    return ''

In [10]:
def valid(a, max_len=0):
    if len(a) > 0 and contain_chinese(a):
        if max_len <= 0:
            return True
        elif len(a) <= max_len:
            return True
    return False

In [ ]:
def insert(a, b):
    cur.execute("""INSERT INTO conversation VALUES
            ('{}', '{}')
            """.format(a.replace("'", "''"), b.replace("'", "''")))

Having more data usually don't hurt. Training on a bigger corpus should be beneficial. OpenSubtitles seems the biggest for now. Another trick to artificially increase the dataset size when creating the corpus could be to split the sentences of each training sample (ex: from the sample Q:Sentence 1. Sentence 2. => A:Sentence X. Sentence Y. we could generate 3 new samples: Q:Sentence 1. Sentence 2. => A:Sentence X., Q:Sentence 2. => A:Sentence X. Sentence Y. and Q:Sentence 2. => A:Sentence X.. Warning: other combinations like Q:Sentence 1. => A:Sentence X. won't work because it would break the transition 2 => X which links the question to the answer)

https://github.com/Conchylicultor/DeepQA

In [ ]:
words = Counter()
keys = {}
a = ''
b = ''
c = ''
d = ''
inserted = 0

def insert_if(question, answer, input_len=12, output_len=20):
    k = question + '----------' + answer
    if valid(question, input_len) and valid(answer, output_len) and k not in keys:
        insert(question, answer)
        keys[k] = 1
        return 1
    return 0
        
for index, line in tqdm(enumerate(lines)):
    words.update(Counter(line))
    # <<<
    a = b
    b = c
    c = d
    d = line
    # a -> b
    question = a
    answer = b
    inserted += insert_if(question, answer)
    """
    # a, b -> c
    question = combine(a, b)
    answer = c
    inserted += insert_if(question, answer)
    # b -> c, d
    question = b
    answer = combine(c, d)
    inserted += insert_if(question, answer)
    # b -> c
    question = b
    answer = c
    inserted += insert_if(question, answer)
    """
    # commit batch
    if inserted != 0 and inserted % 50000 == 0:
        conn.commit()
conn.commit()

6041it [00:00, 60409.34it/s]